
---

Ильясов Антон Евгеньевич 411403

---
The task of this homework is to design a computer vision routine to find objects on a shelve
in a supermarket.

Evaluation criteria for this homework:
1. The computer vision routine should accurately find and return the bounding box
coordinates of the query image in the given input image.
2. The routine should work for any number of query images.
3. The returned bounding boxes should be in the format of [x, y, w, h] where (x, y) is the
top-left corner and (w, h) is the width and height of the bounding box.
4. The code should be readable and properly commented.

In [65]:
import cv2
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm.notebook import tqdm

Read image and convert them to gray

In [66]:
# Загрузите изображение полки и изображение коробки сока
shelf_image = cv2.imread('shelf.jpg')
juice_box_image = cv2.imread('juice_box.jpg')


In [67]:
# Преобразуйте изображения в оттенки серого
shelf_gray = cv2.cvtColor(shelf_image, cv2.COLOR_BGR2GRAY)
juice_box_gray = cv2.cvtColor(juice_box_image, cv2.COLOR_BGR2GRAY)


Extracting keypoints and computing descriptors using SIFT

In [68]:
# Создайте объект SIFT
sift = cv2.SIFT_create()

# Найдите ключевые точки и дескрипторы с помощью SIFT
keypoints_shelf, descriptors_shelf = sift.detectAndCompute(shelf_gray, None)
keypoints_juice_box, descriptors_juice_box = sift.detectAndCompute(juice_box_gray, None)

In [69]:
def matcher(kp1, des1, kp2, des2, threshold):
    
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1,des2, k=2)

    # # Apply ratio test
    # good = []
    # for m,n in matches:
    #     if m.distance < threshold*n.distance:
    #         good.append(m)

    # matches = []
    # for pair in good:
    #     matches.append(list(kp1[pair[0].queryIdx].pt + kp2[pair[0].trainIdx].pt))

    # matches = np.array(matches)
    return matches

In [70]:
matches = matcher(keypoints_juice_box, descriptors_juice_box, keypoints_shelf, descriptors_shelf, 0.5)

In [71]:
# Найдите гомографию между ключевыми точками
src_pts = np.float32([keypoints_juice_box[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints_shelf[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# # Используйте RANSAC для вычисления гомографии
# M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# # Получите размеры изображения коробки сока
# h, w = juice_box_image.shape[:2]

# # Используйте гомографию для проецирования углов изображения коробки сока на изображение полки
# pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
# dst = cv2.perspectiveTransform(pts, M)

# # Нарисуйте ограничивающую рамку вокруг коробки сока на изображении полки
# shelf_image = cv2.polylines(shelf_image, [np.int32(dst)], True, (255, 0, 0), 3, cv2.LINE_AA)

# # Покажите результат
# cv2.imshow('Juice Box Detection', shelf_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

AttributeError: 'tuple' object has no attribute 'queryIdx'